# ESG & Financial Performance Analysis - Part 1: Data Exploration
**by Yonathan Hary Hutagalung**

# 0. Data Importing

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

# 1. Data Loading and Initial Exploration

In [2]:
df = pd.read_csv('raw_data.csv')
df.head()

,CompanyID,CompanyName,Industry,Region,Year,Revenue,ProfitMargin,MarketCap,GrowthRate,ESG_Overall,ESG_Environmental,ESG_Social,ESG_Governance,CarbonEmissions,WaterUsage,EnergyConsumption
0,1,Company_1,Retail,Latin America,2015,459.2,6.0,337.5,NaN,57.0,60.7,33.5,76.8,35577.4,17788.7,71154.7
1,1,Company_1,Retail,Latin America,2016,473.8,4.6,366.6,3.2,56.7,58.9,32.8,78.5,37314.7,18657.4,74629.4
2,1,Company_1,Retail,Latin America,2017,564.9,5.2,313.4,19.2,56.5,57.6,34.0,77.8,45006.4,22503.2,90012.9
3,1,Company_1,Retail,Latin America,2018,558.4,4.3,283.0,-1.1,58.0,62.3,33.4,78.3,42650.1,21325.1,85300.2
4,1,Company_1,Retail,Latin America,2019,554.5,4.9,538.1,-0.7,56.6,63.7,30.0,76.1,41799.4,20899.7,83598.8


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CompanyID          11000 non-null  int64  
 1   CompanyName        11000 non-null  object 
 2   Industry           11000 non-null  object 
 3   Region             11000 non-null  object 
 4   Year               11000 non-null  int64  
 5   Revenue            11000 non-null  float64
 6   ProfitMargin       11000 non-null  float64
 7   MarketCap          11000 non-null  float64
 8   GrowthRate         10000 non-null  float64
 9   ESG_Overall        11000 non-null  float64
 10  ESG_Environmental  11000 non-null  float64
 11  ESG_Social         11000 non-null  float64
 12  ESG_Governance     11000 non-null  float64
 13  CarbonEmissions    11000 non-null  float64
 14  WaterUsage         11000 non-null  float64
 15  EnergyConsumption  11000 non-null  float64
dtypes: float64(11), int64(

In [4]:
df.describe()

,CompanyID,Year,Revenue,ProfitMargin,MarketCap,GrowthRate,ESG_Overall,ESG_Environmental,ESG_Social,ESG_Governance,CarbonEmissions,WaterUsage,EnergyConsumption
count,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000,10000.000000,11000.000000,11000.000000,11000.000000,11000.000000,1.100000e+04,1.100000e+04,1.100000e+04
mean,500.500000,2020.000000,4670.850591,10.900455,13380.622236,4.830370,54.615273,56.416991,55.660582,51.767655,1.271462e+06,5.600442e+05,1.165839e+07
std,288.688113,3.162421,9969.954369,8.758711,39922.870373,9.424787,15.893937,26.767233,23.356152,25.323370,5.067760e+06,1.565686e+06,5.095836e+07
min,1.000000,2015.000000,35.900000,-20.000000,1.800000,-36.000000,6.300000,0.000000,0.000000,0.000000,2.042200e+03,1.021100e+03,5.105500e+03
25%,250.750000,2017.000000,938.775000,5.300000,1098.525000,-1.325000,44.100000,34.700000,37.600000,30.775000,1.228530e+05,6.488467e+04,3.069161e+05
50%,500.500000,2020.000000,1902.300000,10.500000,3096.450000,4.900000,54.600000,55.600000,55.150000,52.100000,2.920734e+05,2.038805e+05,1.221745e+06
75%,750.250000,2023.000000,4342.625000,16.300000,9995.500000,11.000000,65.600000,79.000000,73.800000,73.000000,7.407311e+05,5.251880e+05,5.616437e+06
max,1000.000000,2025.000000,180810.400000,50.000000,865271.700000,38.000000,98.800000,100.000000,100.000000,100.000000,1.741047e+08,5.223142e+07,1.741047e+09


In [5]:
df.isnull().sum()

CompanyID               0
CompanyName             0
Industry                0
Region                  0
Year                    0
Revenue                 0
ProfitMargin            0
MarketCap               0
GrowthRate           1000
ESG_Overall             0
ESG_Environmental       0
ESG_Social              0
ESG_Governance          0
CarbonEmissions         0
WaterUsage              0
EnergyConsumption       0
dtype: int64

unique value in categorical column

In [6]:
print(f"Industries: {df['Industry'].nunique()}")
print(f"Regions: {df['Region'].nunique()}")
print(f"Years: {df['Year'].nunique()}")
print(f"Companies: {df['CompanyID'].nunique()}")

Industries: 9
Regions: 7
Years: 11
Companies: 1000


# 2. Data Cleaning and Preparation

In [7]:
def categorize_esg(score):
    if score > 75:
        return 'Excellent'
    elif 50 <= score <= 75:
        return 'Good'
    elif 25 <= score < 50:
        return 'Average'
    else:
        return 'Poor'

df['ESG_Category'] = df['ESG_Overall'].apply(categorize_esg)

# 3. Distribution Analysis

## 3.1 ESG Distribution map

In [8]:
fig1 = make_subplots(rows=2, cols=2,
                     subplot_titles=('Environmental Score', 'Social Score',
                                   'Governance Score', 'Overall Score'))

fig1.add_trace(go.Histogram(x=df['ESG_Environmental'], name='Environmental',
                           marker_color='#27ae60', nbinsx=30),
              row=1, col=1)
fig1.add_trace(go.Histogram(x=df['ESG_Social'], name='Social',
                           marker_color='#3498db', nbinsx=30),
              row=1, col=2)
fig1.add_trace(go.Histogram(x=df['ESG_Governance'], name='Governance',
                           marker_color='#9b59b6', nbinsx=30),
              row=2, col=1)
fig1.add_trace(go.Histogram(x=df['ESG_Overall'], name='Overall',
                           marker_color='#e74c3c', nbinsx=30),
              row=2, col=2)

fig1.update_layout(height=700, showlegend=False,
                  title_text="ESG Component Score Distributions")
fig1.show()

## 3.2 Financial Metrics

In [9]:
fig2 = make_subplots(rows=2, cols=2,
                     subplot_titles=('Revenue', 'Profit Margin',
                                   'Market Cap', 'Growth Rate'))

fig2.add_trace(go.Histogram(x=df['Revenue'], name='Revenue',
                           marker_color='#16a085', nbinsx=30),
              row=1, col=1)
fig2.add_trace(go.Histogram(x=df['ProfitMargin'], name='Profit Margin',
                           marker_color='#f39c12', nbinsx=30),
              row=1, col=2)
fig2.add_trace(go.Histogram(x=df['MarketCap'], name='Market Cap',
                           marker_color='#8e44ad', nbinsx=30),
              row=2, col=1)
fig2.add_trace(go.Histogram(x=df['GrowthRate'].dropna(), name='Growth Rate',
                           marker_color='#c0392b', nbinsx=30),
              row=2, col=2)

fig2.update_layout(height=700, showlegend=False,
                  title_text="Financial Metrics Distributions")
fig2.show()

## 3.3 Environmental Metrics

In [10]:
fig3 = make_subplots(rows=1, cols=3,
                     subplot_titles=('Carbon Emissions', 'Water Usage',
                                   'Energy Consumption'))

fig3.add_trace(go.Histogram(x=df['CarbonEmissions'], name='Carbon',
                           marker_color='#d35400', nbinsx=30),
              row=1, col=1)
fig3.add_trace(go.Histogram(x=df['WaterUsage'], name='Water',
                           marker_color='#3498db', nbinsx=30),
              row=1, col=2)
fig3.add_trace(go.Histogram(x=df['EnergyConsumption'], name='Energy',
                           marker_color='#f1c40f', nbinsx=30),
              row=1, col=3)

fig3.update_layout(height=400, showlegend=False,
                  title_text="Environmental Metrics Distributions")
fig3.show()

# 4. Correlation Analysis

In [11]:
numeric_cols = ['ESG_Overall', 'ESG_Environmental', 'ESG_Social', 
                'ESG_Governance', 'ProfitMargin', 'Revenue', 'MarketCap',
                'CarbonEmissions', 'WaterUsage', 'EnergyConsumption', 'GrowthRate']
corr_matrix = df[numeric_cols].corr()
fig4 = px.imshow(corr_matrix,
                text_auto='.2f',
                aspect='auto',
                color_continuous_scale='RdBu_r',
                title='Correlation Matrix: ESG and Financial Metrics',
                zmin=-1, zmax=1,
                labels=dict(color="Correlation"))
fig4.update_layout(height=700, width=900)
fig4.show()


In [12]:
esg_corr = corr_matrix['ESG_Overall'].sort_values(ascending=False)
print(esg_corr)

ESG_Overall          1.000000
ESG_Governance       0.671476
ESG_Social           0.662357
ESG_Environmental    0.568171
Revenue              0.148869
MarketCap            0.143793
ProfitMargin         0.088006
GrowthRate           0.068218
CarbonEmissions     -0.039672
WaterUsage          -0.042911
EnergyConsumption   -0.043649
Name: ESG_Overall, dtype: float64


## 4.1 ESG and Profit Margin relationship

In [13]:
fig5 = px.scatter(df.sample(min(1000, len(df))), 
                 x='ESG_Overall', 
                 y='ProfitMargin',
                 color='ESG_Category',
                 color_discrete_map={
                     'Excellent': '#2ecc71',
                     'Good': '#3498db',
                     'Average': '#f39c12',
                     'Poor': '#e74c3c'
                 },
                 title='ESG Score vs Profit Margin',
                 trendline='ols',
                 labels={'ESG_Overall': 'ESG Score',
                        'ProfitMargin': 'Profit Margin (%)'},
                 hover_data=['Industry', 'Region'])
fig5.update_layout(height=600)
fig5.show()

## 4.2 ESG Environmental and Carbon Relationship

In [14]:
fig6 = px.scatter(df.sample(min(1000, len(df))), 
                 x='CarbonEmissions', 
                 y='ESG_Environmental',
                 title='Carbon Emissions vs Environmental Score',
                 trendline='ols',
                 labels={'CarbonEmissions': 'Carbon Emissions (tons CO₂)',
                        'ESG_Environmental': 'Environmental Score'},
                 hover_data=['Region', 'ESG_Overall'])
fig6.update_layout(height=600)
fig6.show()

# 5. Statistical Testing

## 5.1 T-test for comparing profit margins between high and low ESG companies

In [15]:
high_esg = df[df['ESG_Overall'] > df['ESG_Overall'].median()]['ProfitMargin']
low_esg = df[df['ESG_Overall'] <= df['ESG_Overall'].median()]['ProfitMargin']

t_stat, p_value = stats.ttest_ind(high_esg, low_esg)

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

T-statistic: 7.4324
P-value: 0.0000


Result: Statistically significant difference (p < 0.05) → Companies with higher ESG scores show different profit margins

## 5.2 ANOVA Profit Margin across ESG Categories

In [16]:
categories = df.groupby('ESG_Category')['ProfitMargin'].apply(list)
f_stat, p_value_anova = stats.f_oneway(*[categories[cat] for cat in categories.index])

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value_anova:.4f}")

F-statistic: 30.4166
P-value: 0.0000


Result: Statistically significant difference (p < 0.05) meaning ESG category significantly affects profit margin

## 5.3 Correlation significance - ESG vs Revenue

In [17]:
corr_coef, p_value_corr = stats.pearsonr(df['ESG_Overall'].dropna(), 
                                         df['Revenue'].dropna())

print(f"Correlation Coefficient: {corr_coef:.4f}")
print(f"P-value: {p_value_corr:.4f}")

Correlation Coefficient: 0.1489
P-value: 0.0000


Statistically significant correlation (p < 0.05)

# 6. Industry and Regional Analysis

## 6.1 Industry Analysis

### Industry stats

In [18]:
industry_stats = df.groupby('Industry').agg({
    'ESG_Overall': 'mean',
    'ProfitMargin': 'mean',
    'Revenue': 'mean',
    'CarbonEmissions': 'mean',
    'CompanyID': 'count'
}).rename(columns={'CompanyID': 'Company_Count'}).sort_values('ESG_Overall', ascending=False)

In [19]:
industry_stats.round(2)

,ESG_Overall,ProfitMargin,Revenue,CarbonEmissions,Company_Count
Industry,,,,,
Finance,64.62,14.29,6273.96,72050.45,1243
Technology,63.35,18.80,15772.70,450418.73,1045
Healthcare,57.25,15.45,4653.70,335329.19,1331
Retail,55.68,5.49,1613.26,129712.59,1166
Consumer Goods,54.83,10.81,2098.92,340368.45,1276
Utilities,51.60,9.68,1244.21,869341.08,1177
Manufacturing,50.46,8.49,2313.85,623596.85,1287
Energy,49.01,10.55,8799.24,8191264.71,1188
Transportation,46.03,5.37,1125.98,650314.92,1287


### ESG Scoring

In [20]:
fig7 = go.Figure(go.Bar(
    x=industry_stats['ESG_Overall'].sort_values(),
    y=industry_stats['ESG_Overall'].sort_values().index,
    orientation='h',
    marker=dict(color=industry_stats['ESG_Overall'].sort_values(), 
                colorscale='RdYlGn',
                showscale=True,
                colorbar=dict(title="ESG Score"))
))
fig7.update_layout(
    title='Average ESG Score by Industry',
    xaxis_title='Average ESG Score',
    yaxis_title='Industry',
    height=600
)
fig7.show()

### Profit Margin

In [21]:
fig8 = go.Figure(go.Bar(
    x=industry_stats['ProfitMargin'].sort_values(),
    y=industry_stats['ProfitMargin'].sort_values().index,
    orientation='h',
    marker=dict(color=industry_stats['ProfitMargin'].sort_values(), 
                colorscale='Blues',
                showscale=True,
                colorbar=dict(title="Profit Margin %"))
))
fig8.update_layout(
    title='Average Profit Margin by Industry',
    xaxis_title='Average Profit Margin (%)',
    yaxis_title='Industry',
    height=600
)
fig8.show()

## 6.2 Regional Analysis

### Region Stats

In [22]:
region_stats = df.groupby('Region').agg({
    'ESG_Overall': 'mean',
    'ProfitMargin': 'mean',
    'Revenue': 'mean',
    'CarbonEmissions': 'mean',
    'CompanyID': 'count'
}).rename(columns={'CompanyID': 'Company_Count'}).round(2)
print(region_stats)

               ESG_Overall  ProfitMargin  Revenue  CarbonEmissions  \
Region                                                               
Africa               44.51         11.37  4061.01       1134892.19   
Asia                 51.90         11.34  4182.60       1119859.43   
Europe               67.87         10.29  5594.86       1552770.45   
Latin America        50.52         10.38  5666.64       1132198.58   
Middle East          43.44         10.77  4176.84        887571.76   
North America        61.22         11.29  4694.10       1538794.57   
Oceania              62.44         10.85  4398.68       1535759.55   

               Company_Count  
Region                        
Africa                  1463  
Asia                    1672  
Europe                  1540  
Latin America           1507  
Middle East             1617  
North America           1540  
Oceania                 1661  


### Regional ESG comparison

In [23]:
fig9 = go.Figure(go.Bar(
    x=region_stats.index,
    y=region_stats['ESG_Overall'],
    marker=dict(color=region_stats['ESG_Overall'], 
                colorscale='Viridis',
                showscale=True)
))
fig9.update_layout(
    title='Average ESG Score by Region',
    xaxis_title='Region',
    yaxis_title='Average ESG Score',
    height=600
)
fig9.show()

# 7. Temporal Features

In [24]:
yearly_trends = df.groupby('Year').agg({
    'ESG_Overall': 'mean',
    'ESG_Environmental': 'mean',
    'ESG_Social': 'mean',
    'ESG_Governance': 'mean',
    'ProfitMargin': 'mean',
    'Revenue': 'mean',
    'CarbonEmissions': 'mean'
}).round(2)
print(yearly_trends)

      ESG_Overall  ESG_Environmental  ESG_Social  ESG_Governance  \
Year                                                               
2015        51.44              51.91       53.13           49.29   
2016        52.04              52.68       53.57           49.88   
2017        52.68              53.60       54.01           50.41   
2018        53.35              54.58       54.58           50.90   
2019        53.91              55.48       55.03           51.20   
2020        54.59              56.40       55.70           51.66   
2021        55.24              57.26       56.25           52.21   
2022        55.92              58.30       56.72           52.74   
2023        56.59              59.24       57.29           53.23   
2024        57.18              60.11       57.76           53.67   
2025        57.83              61.03       58.22           54.25   

      ProfitMargin  Revenue  CarbonEmissions  
Year                                          
2015         10.88  3

## 7.1 ESG Related Temporal Features

In [25]:
fig10 = go.Figure()
fig10.add_trace(go.Scatter(x=yearly_trends.index, 
                          y=yearly_trends['ESG_Overall'],
                          mode='lines+markers', 
                          name='ESG Overall',
                          line=dict(width=3)))
fig10.add_trace(go.Scatter(x=yearly_trends.index, 
                          y=yearly_trends['ESG_Environmental'],
                          mode='lines+markers', 
                          name='Environmental'))
fig10.add_trace(go.Scatter(x=yearly_trends.index, 
                          y=yearly_trends['ESG_Social'],
                          mode='lines+markers', 
                          name='Social'))
fig10.add_trace(go.Scatter(x=yearly_trends.index, 
                          y=yearly_trends['ESG_Governance'],
                          mode='lines+markers', 
                          name='Governance'))

fig10.update_layout(
    title='ESG Component Trends Over Time (2015-2025)',
    xaxis_title='Year',
    yaxis_title='Score',
    height=600,
    hovermode='x unified'
)
fig10.show()

## 7.2 Financial Related Trend

In [26]:
fig11 = go.Figure()
fig11.add_trace(go.Scatter(x=yearly_trends.index, 
                          y=yearly_trends['ProfitMargin'],
                          mode='lines+markers', 
                          name='Profit Margin',
                          line=dict(width=3, color='green')))
fig11.add_trace(go.Scatter(x=yearly_trends.index, 
                          y=yearly_trends['Revenue']/1000,
                          mode='lines+markers', 
                          name='Revenue (in billions)',
                          yaxis='y2',
                          line=dict(width=3, color='blue')))

fig11.update_layout(
    title='Financial Metrics Trends Over Time (2015-2025)',
    xaxis_title='Year',
    yaxis_title='Profit Margin (%)',
    yaxis2=dict(
        title='Revenue (Billions USD)',
        overlaying='y',
        side='right'
    ),
    height=600,
    hovermode='x unified'
)
fig11.show()

# 8. EDA Summary

**Key Insights from EDA**

Dataset Overview:

- Total companies: 1,000
- Industries covered: 9
- Regions covered: 7
- Years of data: 11 (2015-2025)
- Total records: 11,000

ESG Performance:
- Average ESG Score: 54.62
- Top ESG Industry: Finance
- Bottom ESG Industry: Transportation
- ESG trend: Improving

Financial Performance:
- Average Profit Margin: 10.90%
- Average Revenue: $4,671M
- ESG-Profit Correlation: 0.088

Environmental Impact:
- Average Carbon Emissions: 1,271,462 tons CO₂
- Average Water Usage: 560,044 m³
- Average Energy Consumption: 11,658,394 MWh

In [ ]:
#df.to_csv('esg_processed.csv', index=False)